# 如何在聊天模型中使用少样本示例

:::info 先决条件

本指南假定您已熟悉以下概念：
- [Prompt 模板](/docs/concepts/prompt_templates)
- [示例选择器](/docs/concepts/example_selectors)
- [聊天模型](/docs/concepts/chat_models)
- [向量存储](/docs/concepts/vectorstores)

:::

本指南介绍如何使用示例输入和输出来提示聊天模型。向模型提供几个这样的示例称为[少样本提示](/docs/concepts/few_shot_prompting/)，这是一种简单而强大的指导生成的方法，在某些情况下可以显著提高模型性能。

对于如何进行少样本提示，似乎还没有达成广泛共识，并且最优的提示编译很可能因模型而异。因此，我们提供类似 [FewShotChatMessagePromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.few_shot.FewShotChatMessagePromptTemplate.html?highlight=fewshot#langchain_core.prompts.few_shot.FewShotChatMessagePromptTemplate) 的少样本提示模板，将其作为一个灵活的起点，您可以根据需要对其进行修改或替换。

少样本提示模板的目标是根据输入动态选择示例，然后将这些示例格式化到最终的提示中提供给模型。

**注意：** 以下代码示例仅适用于聊天模型，因为 `FewShotChatMessagePromptTemplates` 被设计为输出格式化的[聊天消息](/docs/concepts/messages)，而不是纯文本字符串。对于与兼容完成模型 (LLM) 的纯文本模板类似的少样本提示示例，请参阅[少样本提示模板](/docs/how_to/few_shot_examples/)指南。

## 固定示例

最基本（也是最常用）的少样本提示技术是使用固定提示示例。这样您就可以选择一个链，对其进行评估，而无需担心生产环境中的其他活动部件。

模板的基本组成部分是：
- `examples`: 要包含在最终提示中的字典示例列表。
- `example_prompt`: 通过其 [`format_messages`](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html?highlight=format_messages#langchain_core.prompts.chat.ChatPromptTemplate.format_messages) 方法将每个示例转换为一个或多个消息。常见的示例是将每个示例转换为一条人类消息和一条 AI 回复消息，或者是一条人类消息后跟一条函数调用消息。

下面是一个简单的演示。首先，定义您希望包含的示例。让我们给 LLM 一个不熟悉的数学运算符，用“🦜”符号表示：

In [1]:
%pip install -qU langchain langchain-openai langchain-chroma

import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass()

如果我们问模型这个表达式的结果是什么，它会失败：

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

model.invoke("What is 2 🦜 9?")

AIMessage(content='The expression "2 🦜 9" is not a standard mathematical operation or equation. It appears to be a combination of the number 2 and the parrot emoji 🦜 followed by the number 9. It does not have a specific mathematical meaning.', response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 17, 'total_tokens': 71}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-aad12dda-5c47-4a1e-9949-6fe94e03242a-0', usage_metadata={'input_tokens': 17, 'output_tokens': 54, 'total_tokens': 71})

现在我们来看看，如果我们给 LLM 一些示例来让它工作，会发生什么。我们将在下面定义一些示例：

In [5]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

examples = [
    {"input": "2 🦜 2", "output": "4"},
    {"input": "2 🦜 3", "output": "5"},
]

接下来，将它们组合成 few-shot prompt 模板。

In [6]:
# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.invoke({}).to_messages())

[HumanMessage(content='2 🦜 2'), AIMessage(content='4'), HumanMessage(content='2 🦜 3'), AIMessage(content='5')]


最后，我们组装最终的提示，如下所示，将 `few_shot_prompt` 直接传递给 `from_messages` 工厂方法，并将其与模型一起使用：

In [7]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

现在，让我们向模型提出初始问题，看看它的表现如何：

In [8]:
from langchain_openai import ChatOpenAI

chain = final_prompt | model

chain.invoke({"input": "What is 2 🦜 9?"})

AIMessage(content='11', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 60, 'total_tokens': 61}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5ec4e051-262f-408e-ad00-3f2ebeb561c3-0', usage_metadata={'input_tokens': 60, 'output_tokens': 1, 'total_tokens': 61})

我们看到模型已经根据给定的少样本示例推断出鹦鹉表情符号代表加法！

## 动态少样本提示

有时，您可能希望根据输入仅从整体集合中选择少量示例进行展示。为此，您可以将传递给 `FewShotChatMessagePromptTemplate` 的 `examples` 替换为 `example_selector`。其他组件与上述相同！我们的动态少样本提示模板将如下所示：

- `example_selector`：负责为给定输入选择少样本示例（以及返回它们的顺序）。这些实现了 [BaseExampleSelector](https://python.langchain.com/api_reference/core/example_selectors/langchain_core.example_selectors.base.BaseExampleSelector.html?highlight=baseexampleselector#langchain_core.example_selectors.base.BaseExampleSelector) 接口。一个常见的例子是基于向量存储的 [SemanticSimilarityExampleSelector](https://python.langchain.com/api_reference/core/example_selectors/langchain_core.example_selectors.semantic_similarity.SemanticSimilarityExampleSelector.html?highlight=semanticsimilarityexampleselector#langchain_core.example_selectors.semantic_similarity.SemanticSimilarityExampleSelector)
- `example_prompt`：通过其 [`format_messages`](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html?highlight=chatprompttemplate#langchain_core.prompts.chat.ChatPromptTemplate.format_messages) 方法将每个示例转换为一个或多个消息。一个常见的例子是将每个示例转换为一条人类消息和一条 AI 回复消息，或者一条人类消息后跟一条函数调用消息。

这些同样可以与其他消息和聊天模板组合，以组装您的最终提示。

让我们通过一个使用 `SemanticSimilarityExampleSelector` 的示例进行探讨。由于此实现使用向量存储根据语义相似性选择示例，因此我们首先需要填充该存储。这里的基本思想是，我们希望搜索并返回与文本输入最相似的示例，因此我们嵌入提示示例的 `values` 而不是考虑键：

In [9]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

examples = [
    {"input": "2 🦜 2", "output": "4"},
    {"input": "2 🦜 3", "output": "5"},
    {"input": "2 🦜 4", "output": "6"},
    {"input": "What did the cow say to the moon?", "output": "nothing at all"},
    {
        "input": "Write me a poem about the moon",
        "output": "One for the moon, and one for me, who are we to talk about the moon?",
    },
]

to_vectorize = [" ".join(example.values()) for example in examples]
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

### 创建 `example_selector`

创建了 vectorstore 后，我们就可以创建 `example_selector`。在这里，我们将单独调用它，并设置其 `k` 值以仅检索最接近输入的两个示例。

In [10]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

# The prompt template will load examples by passing the input do the `select_examples` method
example_selector.select_examples({"input": "horse"})

[{'input': 'What did the cow say to the moon?', 'output': 'nothing at all'},
 {'input': '2 🦜 4', 'output': '6'}]

### 创建 Prompt 模板

现在，我们使用上面创建的 `example_selector` 来组装 Prompt 模板。

In [11]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# Define the few-shot prompt.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    # The input variables select the values to pass to the example_selector
    input_variables=["input"],
    example_selector=example_selector,
    # Define how each example will be formatted.
    # In this case, each example will become 2 messages:
    # 1 human, and 1 AI
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

print(few_shot_prompt.invoke(input="What's 3 🦜 3?").to_messages())

[HumanMessage(content='2 🦜 3'), AIMessage(content='5'), HumanMessage(content='2 🦜 4'), AIMessage(content='6')]


然后我们可以将这个few-shot聊天消息提示模板传递到另一个聊天提示模板中：

In [12]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

print(few_shot_prompt.invoke(input="What's 3 🦜 3?"))

messages=[HumanMessage(content='2 🦜 3'), AIMessage(content='5'), HumanMessage(content='2 🦜 4'), AIMessage(content='6')]


### 与聊天模型配合使用

最后，您可以将模型连接到少样本提示。

In [13]:
chain = final_prompt | ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

chain.invoke({"input": "What's 3 🦜 3?"})

AIMessage(content='6', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 60, 'total_tokens': 61}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d1863e5e-17cd-4e9d-bf7a-b9f118747a65-0', usage_metadata={'input_tokens': 60, 'output_tokens': 1, 'total_tokens': 61})

## 后续步骤

您现在已经学会了如何在聊天提示中添加少样本示例。

接下来，请查看本节中关于提示模板的其他操作指南，相关的关于使用文本补全模型进行少样本操作的操作指南 [few shotting with text completion models](/docs/how_to/few_shot_examples)，或者其他的 [示例选择器操作指南](/docs/how_to/example_selectors/)。